In [2]:
import numpy as np
import pandas as pd

In [3]:
num_samples = 10000
latent_dim = 256
batch_size = 64  
input_texts = []
target_texts = []
epochs = 100  
input_characters = set()
target_characters = set()

In [4]:
with open("spa.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

In [5]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    
    ############### A ###############
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    ############### B ###############
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
print(input_characters)

print("\n")

print(target_characters)

{'W', '5', 'p', 'L', 'q', ',', 'v', '7', '9', 'I', 'c', 'm', 't', 'G', 'D', '4', 'R', 'h', 'K', 'V', 'k', '?', 'M', 'E', '.', 'C', 'e', 'b', 'n', 'l', '!', 'u', 'N', 'y', 'f', 'B', 'a', 'j', 'F', 'H', 'P', '3', 'g', 'o', 'x', 'T', 'w', '1', 'z', 'd', 'Y', 'J', 's', "'", 'U', ':', 'Q', '6', 'O', ' ', '$', '0', 'S', 'A', 'r', 'i', '2', '8', '-'}


{'"', 'C', 'e', 'l', '!', 'u', 'B', 'a', 'j', '3', 'x', 'z', 'd', "'", 'ó', ' ', 'S', 'A', 'i', 'í', 'W', 'L', 'q', 'v', '7', 'I', 't', '\t', 'V', '?', '.', 'n', 'N', 'F', '1', 'Á', 's', 'U', ':', 'O', 'É', '2', '8', ',', '»', '4', 'G', 'h', 'k', 'M', 'E', 'b', 'P', 'g', 'w', 'Q', 'J', 'Ú', 'r', 'ú', '-', '5', 'p', 'ü', 'é', 'á', 'c', '\n', 'm', 'D', 'ñ', 'Ó', 'R', '¡', 'y', 'f', 'H', 'o', 'T', 'Y', '¿', '6', '«', '0'}


In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) # 69
num_decoder_tokens = len(target_characters) # 84
max_encoder_seq_length = max([len(txt) for txt in input_texts]) #16
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # 78

print("No.of samples:", len(input_texts))
print("No.of unique input tokens:", num_encoder_tokens)
print("No.of unique output tokens:", num_decoder_tokens)
print("Maximum seq length for inputs:", max_encoder_seq_length)
print("Maximum seq length for outputs:", max_decoder_seq_length)

No.of samples: 10000
No.of unique input tokens: 69
No.of unique output tokens: 84
Maximum seq length for inputs: 16
Maximum seq length for outputs: 78


In [7]:
print(input_characters)

print("\n")

print(target_characters)

[' ', '!', '$', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


['\t', '\n', ' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Ó', 'Ú', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü']


In [8]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print(input_token_index)

print("\n")

print(target_token_index)

{' ': 0, '!': 1, '$': 2, "'": 3, ',': 4, '-': 5, '.': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '5': 12, '6': 13, '7': 14, '8': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68}


{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 4

In [19]:
len(input_texts)

10000

In [20]:
max_encoder_seq_length

16

In [21]:
num_encoder_tokens

69

In [9]:
encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)


In [10]:
encoder_input_data.shape # (Samples(Input_Text),Max_seq_Length_Input_text,Features)

(10000, 16, 69)

In [11]:
decoder_input_data.shape # (Samples(Target_Text),Max_seq_Length_Target_text,Features)

(10000, 78, 84)

In [12]:
decoder_target_data.shape # (Samples(Target_Text),Max_seq_Length_Target_text,Features)

(10000, 78, 84)

In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [14]:
import keras

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
encoder_input_data.shape

(10000, 16, 69)

In [17]:
decoder_input_data.shape

(10000, 78, 84)

In [18]:
decoder_target_data.shape

(10000, 78, 84)

In [124]:
from keras.optimizers import Adam

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=["accuracy"])

model.fit([encoder_input_data,
           decoder_input_data],
           decoder_target_data,
           batch_size=3,
           epochs=epochs,validation_split=0.2,)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 67s 8ms/step - loss: 0.8149 - accuracy: 0.8112 - val_loss: 0.5507 - val_accuracy: 0.8380
Epoch 2/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.4360 - accuracy: 0.8713 - val_loss: 0.4867 - val_accuracy: 0.8550
Epoch 3/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3925 - accuracy: 0.8830 - val_loss: 0.5562 - val_accuracy: 0.8394
Epoch 4/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3898 - accuracy: 0.8837 - val_loss: 0.4415 - val_accuracy: 0.8646
Epoch 5/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3517 - accuracy: 0.8937 - val_loss: 0.4187 - val_accuracy: 0.8728
Epoch 6/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3344 - accuracy: 0.8986 - val_loss: 0.4055 - va

In [125]:
model.save("E2S")

In [128]:
from keras.utils import plot_model
#plot_model(model, to_file='modelsummary.png', show_shapes=True, show_layer_names=True)

In [129]:
# Decode the Sentence

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc] # encoder hidden states
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [131]:
# Now build the model for the decoder

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [133]:
# Create two reverse-lookup token indexes to decode the sequence to make it readable

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [155]:
print(reverse_input_char_index)

{0: ' ', 1: '!', 2: '$', 3: "'", 4: ',', 5: '-', 6: '.', 7: '0', 8: '1', 9: '2', 10: '3', 11: '4', 12: '5', 13: '6', 14: '7', 15: '8', 16: '9', 17: ':', 18: '?', 19: 'A', 20: 'B', 21: 'C', 22: 'D', 23: 'E', 24: 'F', 25: 'G', 26: 'H', 27: 'I', 28: 'J', 29: 'K', 30: 'L', 31: 'M', 32: 'N', 33: 'O', 34: 'P', 35: 'Q', 36: 'R', 37: 'S', 38: 'T', 39: 'U', 40: 'V', 41: 'W', 42: 'Y', 43: 'a', 44: 'b', 45: 'c', 46: 'd', 47: 'e', 48: 'f', 49: 'g', 50: 'h', 51: 'i', 52: 'j', 53: 'k', 54: 'l', 55: 'm', 56: 'n', 57: 'o', 58: 'p', 59: 'q', 60: 'r', 61: 's', 62: 't', 63: 'u', 64: 'v', 65: 'w', 66: 'x', 67: 'y', 68: 'z'}


In [156]:
print(reverse_target_char_index)

{0: '\t', 1: '\n', 2: ' ', 3: '!', 4: '"', 5: "'", 6: ',', 7: '-', 8: '.', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: ':', 19: '?', 20: 'A', 21: 'B', 22: 'C', 23: 'D', 24: 'E', 25: 'F', 26: 'G', 27: 'H', 28: 'I', 29: 'J', 30: 'L', 31: 'M', 32: 'N', 33: 'O', 34: 'P', 35: 'Q', 36: 'R', 37: 'S', 38: 'T', 39: 'U', 40: 'V', 41: 'W', 42: 'Y', 43: 'a', 44: 'b', 45: 'c', 46: 'd', 47: 'e', 48: 'f', 49: 'g', 50: 'h', 51: 'i', 52: 'j', 53: 'k', 54: 'l', 55: 'm', 56: 'n', 57: 'o', 58: 'p', 59: 'q', 60: 'r', 61: 's', 62: 't', 63: 'u', 64: 'v', 65: 'w', 66: 'x', 67: 'y', 68: 'z', 69: '¡', 70: '«', 71: '»', 72: '¿', 73: 'Á', 74: 'É', 75: 'Ó', 76: 'Ú', 77: 'á', 78: 'é', 79: 'í', 80: 'ñ', 81: 'ó', 82: 'ú', 83: 'ü'}


In [333]:
def decode_sequence(input_seq):
    print('input_seq:')
    print(input_seq)
    states_value = encoder_model.predict(input_seq)
    print("Predicted states value shape:",states_value[0].shape)
    print("Predicted states value:")
    print(states_value)
    

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        print(output_tokens)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print(sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        print(sampled_char)
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [334]:
#i = np.random.choice(len(input_texts))


input_seq = encoder_input_data[i:i+1]
translation = decode_sequence(input_seq)
print('-')
print('Input:', input_texts[i])
print('Translation:', translation)

input_seq:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
Predicted states value shape: (1, 256)
Predicted states value:
[array([[-1.04037926e-01, -4.05267673e-03, -2.36718237e-01,
         6.13312781e-01, -1.61019295e-01,  2.38609910e-01,
        -2.37695320e-04,  4.79686558e-02, -9.68596160e-01,
         2.56890029e-01,  1.02380633e-01, -9.74036157e-01,
         4.61636577e-04, -2.87840085e-04, -1.42543912e-02,
         4.25163656e-04, -1.11100664e-02, -3.27027887e-01,
         5.31973600e-01, -1.49407893e-01, -3.24678188e-03,
        -1.38657987e-02, -7.18405783e-01,  1.11014444e-04,
         1.96326990e-02,  9.97267067e-01, -4.18721591e-09,
         4.31703895e-01,  9.32199419e-01, -2.00840615e-04,
        -3.86660784e-01,  2.95235366e-01, -3.37293022e-03,
        -4.21886504e-01, -7.72998929e-01,  4.09359902e-01,
         9.91512179e-01, -1.03226176e-03,  1.77263487

In [279]:
input_seq

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [288]:
encoder_model.predict(input_seq)

[array([[-8.65997255e-01, -1.64083466e-01, -1.76009089e-01,
          5.79825044e-01,  1.59735009e-01,  6.97121501e-01,
         -4.42345081e-05,  2.74505198e-01, -9.34010804e-01,
          8.00207734e-01,  4.95174080e-02, -5.22061825e-01,
          8.51509321e-05, -2.08945101e-04, -2.13983655e-02,
          6.32351905e-04, -3.03273648e-02, -9.36433136e-01,
         -2.24891633e-01, -4.97152619e-02, -4.46590036e-03,
         -1.63370669e-02, -1.00597292e-01,  2.18334681e-05,
          3.06617349e-01,  9.98623669e-01, -1.57105760e-08,
         -2.04298332e-01,  7.96407342e-01, -6.92792761e-04,
         -4.36697870e-01,  1.89555492e-02,  1.88346226e-02,
         -4.89849925e-01, -5.03361583e-01,  2.32205003e-01,
          1.09100647e-01, -3.76918833e-05,  4.80260860e-05,
          9.20531154e-02,  2.69680610e-03, -3.10006738e-03,
          1.12479925e-03, -9.75897014e-02,  9.99702871e-01,
         -2.41952538e-02,  7.16429770e-01, -1.69187784e-04,
          8.95674288e-01,  4.03081439e-0

In [289]:
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, target_token_index["\t"]] = 1.0

In [292]:
target_seq

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]]])

In [247]:
names = ["santhosh","kumar","anand","joshi"] # Samples

import string 

# get the alphabest
string.ascii_lowercase

# List of Vocabulary
vocabs_ = [x for x in string.ascii_lowercase] # Features

# vocabulary length
vocab_length = len(string.ascii_lowercase)

# Max Sequence Length
max_seq_length = max([len(x) for x in names])

char_2_int = {} 
int_2_char = {}

# Char to Interger 
char_2_int = dict([(x,i) for i,x in enumerate(vocabs_)])

# Integer to Char
int_2_char = dict([(i,x) for i,x in enumerate(vocabs_)])

# Create Zero array for encoding

encoding = np.zeros((len(names),max_seq_length,vocab_length))

print(encoding.shape)  # (4 Samples,max length of samples,vocabulary Length

# Update the encoder matrix for the samples

for i,name in enumerate(names): # i = index value and i = sample and j =time step
    for j,char in enumerate(name):
        encoding[i,j,char_2_int[char]] = 1.0
        
encoding

(4, 8, 26)


array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0

In [263]:
# Decoder

In [269]:
pedicted_Seq = [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]

In [270]:
pedicted_Seq = np.asarray([np.asarray(pedicted_Seq)])

In [319]:
list(pedicted_Seq[0,2]).index(1)

18

In [320]:
int_2_char[18]

's'